# 1. 데이터 수집

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path_to_dataset = '/content/drive/MyDrive/Colab Notebooks/E-15/ChatbotData .csv'

In [4]:
df = pd.read_csv(path_to_dataset, sep=",")

In [5]:
df

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


---------------------------------------

# 2. 데이터 전처리

#### 하나의 csv 파일에 질문 답변이 모두 저장되어 있으니 사전 작업이 필요

In [6]:
sd1 = df['Q']

In [7]:
sd1

0                         12시 땡!
1                    1지망 학교 떨어졌어
2                   3박4일 놀러가고 싶다
3                3박4일 정도 놀러가고 싶다
4                        PPL 심하네
                  ...           
11818             훔쳐보는 것도 눈치 보임.
11819             훔쳐보는 것도 눈치 보임.
11820                흑기사 해주는 짝남.
11821    힘든 연애 좋은 연애라는게 무슨 차이일까?
11822                 힘들어서 결혼할까봐
Name: Q, Length: 11823, dtype: object

In [8]:
sd1_df = pd.DataFrame(sd1)
sd1_df

,Q
0,12시 땡!
1,1지망 학교 떨어졌어
2,3박4일 놀러가고 싶다
3,3박4일 정도 놀러가고 싶다
4,PPL 심하네
...,...
11818,훔쳐보는 것도 눈치 보임.
11819,훔쳐보는 것도 눈치 보임.
11820,흑기사 해주는 짝남.
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?


---------------------------------------

In [9]:
sd2 = df['A']

In [10]:
sd2

0                      하루가 또 가네요.
1                       위로해 드립니다.
2                     여행은 언제나 좋죠.
3                     여행은 언제나 좋죠.
4                      눈살이 찌푸려지죠.
                   ...           
11818          티가 나니까 눈치가 보이는 거죠!
11819               훔쳐보는 거 티나나봐요.
11820                      설렜겠어요.
11821    잘 헤어질 수 있는 사이 여부인 거 같아요.
11822          도피성 결혼은 하지 않길 바라요.
Name: A, Length: 11823, dtype: object

In [11]:
sd2_df = pd.DataFrame(sd2)
sd2_df

,A
0,하루가 또 가네요.
1,위로해 드립니다.
2,여행은 언제나 좋죠.
3,여행은 언제나 좋죠.
4,눈살이 찌푸려지죠.
...,...
11818,티가 나니까 눈치가 보이는 거죠!
11819,훔쳐보는 거 티나나봐요.
11820,설렜겠어요.
11821,잘 헤어질 수 있는 사이 여부인 거 같아요.


---------------------------------------

#### question, answer 각각 텍스트 파일로 만들기

In [18]:
sd1_df.to_csv('questions.txt', index=False, header=None)

In [21]:
sd2_df.to_csv('answers.txt', index=False, header=None)

In [20]:
path_to_questions = '/content/questions.txt'
print(path_to_questions)

/content/questions.txt


In [23]:
path_to_answers = '/content/answers.txt'
print(path_to_answers)

/content/answers.txt




---



#### 정제

In [55]:
import glob
txt_list1 = glob.glob(path_to_questions)
txt_list2 = glob.glob(path_to_answers)

In [42]:
raw_corpus_q = []

for txt_file in txt_list1:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus_q.extend(raw)

print("데이터 크기:", len(raw_corpus_q))
print("Examples:\n", raw_corpus_q[:3])

데이터 크기: 11823
Examples:
 ['12시 땡!', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다']


In [57]:
raw_corpus_a = []

for txt_file in txt_list2:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus_a.extend(raw)

print("데이터 크기:", len(raw_corpus_a))
print("Examples:\n", raw_corpus_a[:3])

데이터 크기: 11823
Examples:
 ['하루가 또 가네요.', '위로해 드립니다.', '여행은 언제나 좋죠.']


In [61]:
# 전처리 함수
# 토크나이징에 방해되지 않기 위해
def preprocess_sentence(review):
    
  review = review.lower() #lower case 
  review = re.sub(r'\s+', ' ', review) #remove extra space 
  review = re.sub(r'<[^>]+>','',review) #remove Html tags 
  review = re.sub(r'\s+', ' ', review) #remove spaces 
  review = re.sub(r"^\s+", '', review) #remove space from start 
  review = re.sub(r'\s+$', '', review) #remove space from the end
  review = re.sub(r"([?.!,])", r" \1 ", review)
  review = re.sub(r'[" "]+', " ", review)

  return review

In [63]:
print(preprocess_sentence("안녕, 이거 도대체 뭐야."))

안녕 , 이거 도대체 뭐야 . 


In [75]:
corpus_q = []

for sentence in raw_corpus_q:

    preprocessed_sentence = preprocess_sentence(sentence)
    corpus_q.append(preprocessed_sentence)
        
corpus_q[0:20]

['12시 땡 ! ',
 '1지망 학교 떨어졌어',
 '3박4일 놀러가고 싶다',
 '3박4일 정도 놀러가고 싶다',
 'ppl 심하네',
 'sd카드 망가졌어',
 'sd카드 안돼',
 'sns 맞팔 왜 안하지ㅠㅠ',
 'sns 시간낭비인 거 아는데 매일 하는 중',
 'sns 시간낭비인데 자꾸 보게됨',
 'sns보면 나만 빼고 다 행복해보여',
 '가끔 궁금해',
 '가끔 뭐하는지 궁금해',
 '가끔은 혼자인게 좋다',
 '가난한 자의 설움',
 '가만 있어도 땀난다',
 '가상화폐 쫄딱 망함',
 '가스불 켜고 나갔어',
 '가스불 켜놓고 나온거 같아',
 '가스비 너무 많이 나왔다 . ']

In [76]:
corpus_a = []

for sentence in raw_corpus_a:

    preprocessed_sentence = preprocess_sentence(sentence)
    corpus_a.append(preprocessed_sentence)
        
corpus_a[0:20]

['하루가 또 가네요 . ',
 '위로해 드립니다 . ',
 '여행은 언제나 좋죠 . ',
 '여행은 언제나 좋죠 . ',
 '눈살이 찌푸려지죠 . ',
 '다시 새로 사는 게 마음 편해요 . ',
 '다시 새로 사는 게 마음 편해요 . ',
 '잘 모르고 있을 수도 있어요 . ',
 '시간을 정하고 해보세요 . ',
 '시간을 정하고 해보세요 . ',
 '자랑하는 자리니까요 . ',
 '그 사람도 그럴 거예요 . ',
 '그 사람도 그럴 거예요 . ',
 '혼자를 즐기세요 . ',
 '돈은 다시 들어올 거예요 . ',
 '땀을 식혀주세요 . ',
 '어서 잊고 새출발 하세요 . ',
 '빨리 집에 돌아가서 끄고 나오세요 . ',
 '빨리 집에 돌아가서 끄고 나오세요 . ',
 '다음 달에는 더 절약해봐요 . ']

In [66]:
print('전체 샘플 수 :', len(corpus_q))
print('전체 샘플 수 :', len(corpus_a))

전체 샘플 수 : 11823
전체 샘플 수 : 11823


In [67]:
print('전처리 후의 22번째 질문 샘플: {}'.format(corpus_q[21]))
print('전처리 후의 22번째 답변 샘플: {}'.format(corpus_a[21]))

전처리 후의 22번째 질문 샘플: 가스비 장난 아님
전처리 후의 22번째 답변 샘플: 다음 달에는 더 절약해봐요 . 




---



# 3. SubwordTextEncoder 사용

#### 병렬 데이터 전처리

Vocabulary 생성!!!

In [77]:
import tensorflow_datasets as tfds #Tokenizer 생성

tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(corpus_q + corpus_a, target_vocab_size=2**13)

In [78]:
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1] #시작 종료 토큰 넘버 부여

In [80]:
print('START_TOKEN의 번호 :' ,[tokenizer.vocab_size])
print('END_TOKEN의 번호 :' ,[tokenizer.vocab_size + 1])

START_TOKEN의 번호 : [8171]
END_TOKEN의 번호 : [8172]


In [81]:
VOCAB_SIZE = tokenizer.vocab_size + 2 #시작 종료 토큰 추가하여 Vocabulary size 설정
print(VOCAB_SIZE)

8173


Encoding and Padding !!!

In [83]:
# 인코딩해서 단어를 정수로 변환!!!
print('정수 인코딩 후의 21번째 질문 샘플: {}'.format(tokenizer.encode(corpus_q[21])))
print('정수 인코딩 후의 21번째 답변 샘플: {}'.format(tokenizer.encode(corpus_a[21])))

정수 인코딩 후의 21번째 질문 샘플: [5760, 610, 2489, 4160]
정수 인코딩 후의 21번째 답변 샘플: [2353, 7511, 7, 6273, 96, 1]


In [84]:
# 인코딩 후 최대 길이 지정!!!
MAX_LENGTH = 40
print(MAX_LENGTH)

40


In [85]:
# 문장 형식 지정 및 최대 길이로 필터링 및 패딩까지 처리하는 함수
def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []
  
  for (sentence1, sentence2) in zip(inputs, outputs):
    sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
    sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

    if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:
      tokenized_inputs.append(sentence1)
      tokenized_outputs.append(sentence2)
  
  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')
  
  return tokenized_inputs, tokenized_outputs

In [86]:
# 필터링 함수 적용
questions, answers = tokenize_and_filter(corpus_a, corpus_q)
print('단어장의 크기 :',(VOCAB_SIZE))
print('필터링 후의 질문 샘플 개수: {}'.format(len(questions)))
print('필터링 후의 답변 샘플 개수: {}'.format(len(answers)))

단어장의 크기 : 8173
필터링 후의 질문 샘플 개수: 11823
필터링 후의 답변 샘플 개수: 11823


#### 교사 강요(Teacher Forcing) 언어 모델 훈련 기법 (훈련 속도 증강 위해 사용)
- 트랜스포머 모델의 Decoder가 자기 회귀 모델이기 때문에 사용해야 함

In [88]:
# tf.data.Dataset API 입력으로 질문 답변 쌍을 사용함
# 교사 강요, Decoder의 입력값과 레이블을 규칙에 따라 설정함

BATCH_SIZE = 64
BUFFER_SIZE = 20000

dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions,
        'dec_inputs': answers[:, :-1]
    },
    {
        'outputs': answers[:, 1:]
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

# 4. 모델 구성

# 5. 모델 평가

# 회고

- 노드에는 영어 데이터, 데이터 파일의 형태가 다른 것으로 학습을 했는데, 갑자기 한글 데이터 및 다른 형태의 파일로 전처리를 해야해서 시간이 많이 걸렸다.
- 영어와 한글은 다르게 처리해야 한다는 것을 알았다.

[Transformer와 RNN의 차이점]
- RNN은 어순대로 모델에 입력되기 때문에 어순 정보가 필요 없음
- Transformer는 한꺼번에 문장을 입력받기 때문에 어순을 알려줘야 함
- 각 단어에 임베딩 벡터 + 위치정보 벡터 값(사인 함수, 코사인 함수 값)을 더해줘야 함

[Attention이 하는 일]
- Encoder Self Attention : 입력 문장 단어들 간의 유사도 구함
- Decoder Self Attention : 먼저 생성된 단어들과 유사도 구함
- Encoder Decoder Attention : 인코더 입력 단어들과 유사도 구함

[Padding Masking]
- 길이를 맞추기 위해서 빈 공간을 0으로 채우는데, 이 숫자 0을 체크해서 0인 위치는 벡터 1값으로, 문자는 벡터 0 출력

[Look-ahead masking]
- 트랜스포머는 문장을 한번에 들어가는데, 이때 올바른 예측 훈련을 위해 기준이 되는 단어보다 다음에 나오는 단어를 참고하지 않도록 함